In [1]:
# import libraries
from platform import python_version
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
#
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
# seed
RANDOM_STATE = 1776

# set seaborn theme
sns.set_theme()

# print versions
print("Numpy Version: " + np.__version__)
print("Pandas Version: " + pd.__version__)
print("Seaborn Version: " + sns.__version__)
print("Matplotlib Version: " + plt.matplotlib.__version__)
print("Python Version: " + python_version())

# adjust pandas display options to max
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# adjust pandas display options to ensure full display of content
pd.set_option('display.max_colwidth', None)

Numpy Version: 1.26.4
Pandas Version: 2.2.3
Seaborn Version: 0.13.2
Matplotlib Version: 3.9.2
Python Version: 3.9.20


### Read Dataset

In [2]:
# full dataframe
df = pd.read_pickle("../Data/Cleaned_Data.pkl")

# data dictionary
df_dict = pd.read_pickle("../Data/FinalcolumnDefinition.pkl")

# data label
df_label = pd.read_pickle("../Data/Label_Analysis.pkl") 

# remove columns
df_remove = pd.read_pickle("../Data/Remove_Analysis.pkl")

# new columns
df_new = pd.read_pickle("../Data/New_Analysis.pkl")

# don columns
df_don = pd.read_pickle("../Data/DON_Analysis.pkl")

# can columns
df_can = pd.read_pickle("../Data/CAN_Analysis.pkl")

# both columns
df_both = pd.read_pickle("../Data/Both_Analysis.pkl")

# unkown columns
df_unknown = pd.read_pickle("../Data/Unknown_Analysis.pkl")

In [3]:
df.head()

,PreviousTransplantNumber_CAN,WaitListDiagnosisCode_CAN,Gender_CAN,BloodGroup_CAN,Citizenship_CAN,ResidencyState_CAN,EducationLevel_CAN,LifeSupport_ECMO_CAN,LifeSupport_IABP_CAN,IntropesIV_CAN,LifeSupport_PGE_CAN,LifeSupport_OTHER_CAN,VentricularDeviceType_CAN,VentricularDeviceBrand_CAN,FunctionalStatus_CAN,PrimaryPaymentSource_CAN,PrimaryDiagnosisCode_CAN,DiabetesType_CAN,DialysisType_CAN,CerebrovascularDisease_CAN,PreviousMalignancy_CAN,Creatinine_CAN,DefibrillatorImplant_CAN,Hemodynamics_SYS_CAN,Hemodynamics_PA_DIA_CAN,Hemodynamics_PA_MN_CAN,Hemodynamics_PCW_CAN,Hemodynamics_CO_CAN,IntropesVasodilators_SYS_CAN,IntropesVasodilators_DIA_CAN,IntropesVasodilators_MN_CAN,IntropesVasodilators_PCW_CAN,IntropesVasodilators_CO_CAN,CigaretteUse_CAN,CigaretteAbstinence_CAN,PriorCardiacSurgery_CAN,PriorCardiacSurgeryType_CAN,TotalDayWaitList_CAN,LifeSupport_CAN,Hispanic_CAN,Ethnicity_CAN,Ventilator_CAN,TransplantRegion,WorkIncome_CAN,AntigenBW4_CAN,AntigenBW6_CAN,AntigenC1_CAN,AntigenC2_CAN,AntigenDR51_CAN,AntigenDR51_2_CAN,AntigenDR52_CAN,AntigenDR52_2_CAN,AntigenDR53_CAN,AntigenDR53_2_CAN,AntigenDQ1_CAN,AntigenDQ2_CAN,FunctionalStatusAfter_CAN,PreTransplantMedicalCondition_CAN,StatusAfter_CAN,PrimaryPaymentSourceAfter_CAN,LifeSupportAfter_ECMO_CAN,ResidencyStateAfter_CAN,WorkIncomeAfter_CAN,LifeSupportAfter_PGE_CAN,CreatinineAfter_CAN,DialysisAfterList_CAN,HemodynamicsAfter_CO_CAN,HemodynamicsAfter_PA_DIA_CAN,HemodynamicsAfter_PA_MN_CAN,HemodynamicsAfter_PCW_CAN,HemodynamicsAfter_SYS_CAN,LifeSupportAfter_IABP_CAN,InfectionTherapyIV_CAN,IntropesIVAfter_CAN,IntropesVasodilatorsAfter_CO_CAN,IntropesVasodilatorsAfter_DIA_CAN,IntropesVasodilatorsAfter_MN_CAN,IntropesVasodilatorsAfter_PCW_CAN,IntropesVasodilatorsAfter_SYS_CAN,LifeSupportAfter_OTHER_CAN,PriorLungSurgery_CAN,AirwayDehiscencePostTransplant_CAN,AcuteRejectionEpisode_CAN,StrokePostTransplant_CAN,DialysisPriorDischarge_CAN,PacemakerPostTransplant_CAN,ChronicSteroidsUse_CAN,TotalBilirubin_CAN,Transfusion_CAN,VentricularDeviceTypeAfter_CAN,VentricularDeviceBrandAfter_CAN,VentilatorAfter_CAN,LifeSupportInhaledAfter_CAN,PriorCardiacSurgeryTypeAfter_CAN,Hepatitis_B_CoreAntibody_CAN,SurfaceAntigenHEP_B_CAN,AtTransplantSurfaceHBVAntibodyTotal_CAN,CMVStatusAtTransplant_CAN,TransplantSerostatusHIV_CAN,SerostatusStatus_HEP_C_CAN,EpsteinBarrStatusTransplant_CAN,PreTransplantResultHIV_NAT_CAN,PreTranspantHCV_NAT_CAN,ResultHBVAfter_NAT_CAN,GraftFailed_CAN,LastFollowupNumber_CAN,TransplantStatus_CAN,TransplantSurvivalDay_CAN,RecipientStatus_CAN,FollowUpFunctionalStatus_CAN,Status_CAN,LifeSupportInhaled_CAN,DeceasedRetyped_DON,CrossMatch,Recent_CPRA_CAN,Peak_CPRA_CAN,RejectionTreatmentWithinOneYear_CAN,PreviousTransplantSameOrgan_CAN,PreviousTransplantAnyOrgan_CAN,AntigenHistocompatibility_A1_DON,AntigenHistocompatibility_A2_DON,AntigenHistocompatibility_B1_DON,AntigenHistocompatibility_B2_DON,AntigenHistocompatibility_DR1_DON,AntigenHistocompatibility_DR2_DON,AntigenHLAcompatibility_A1_CAN,AntigenHLAcompatibility_A2_CAN,AntigenHLAcompatibility_B1_CAN,AntigenHLAcompatibility_B2_CAN,AntigenHLAcompatibility_DR1_CAN,AntigenHLAcompatibility_DR2_CAN,MismatchLevel_LOCUS,MismatchLevel_B_LOCUS,MismatchLevel_DR_LOCUS,MismatchLevel_HLA,Citizenship_DON,PastCocaineUse_DON,AgeCalc_DON,Ethnicity_DON,Hepatitis_B_CoreAntibody_DON,SurfaceAntigenHEP_B_DON,BloodGroup_DON,HeavyAlcoholUse_DON,Gender_DON,HomeState_DON,Antibody_HEP_C_DON,NonHeartBeating_DON,Antihypertensive_DON,BloodInfectionSource_DON,BloodUreaNitrogenLevel_DON,Creatinine_DON,OtherInfectionSource_DON,Diuretics_DON,SteroidsUse_DON,Triiodothyronine_T3_DON,Thyroxine_T4_DON,PulmonaryInfection_DON,Level_SGOT_AST_DON,Level_SGOT_ALT_DON,TotalBilirubin_DON,UrineInfection_DON,Vasodilator_DON,Result_RPR_VDRL_DON,ClinicalInfection_DON,CocaineUse_DON,OtherDrugUse_DON,ExtracanialCancer_DON,CancerSite_DON,CigaretteHistory_DON,Hypertension_DON,IntracanialCancer_DON,HistoryCancer_DON,DiabetesHistory_DON,Diabetes_DON,PastOtherDrugUse_DON,SkinCancer_DON,SerologyAnti

In [4]:
# df DataFrame
print(f"Orginal Data file Shape: {df.shape}")
print(f"Label(s) Data rows: {len(df_label)}")
print(f"Remove Column(s) Data rows: {len(df_remove)}")
print(f"Donor Column(s) Data rows: {len(df_don)}")
print(f"Candidate Column(s) Data rows: {len(df_can)}")
print(f"Both Column(s) Data rows: {len(df_both)}")
print(f"Unknown Column(s) Data rows: {len(df_unknown)}")

Orginal Data file Shape: (15282, 286)
Label(s) Data rows: 12
Remove Column(s) Data rows: 92
Donor Column(s) Data rows: 87
Candidate Column(s) Data rows: 172
Both Column(s) Data rows: 23
Unknown Column(s) Data rows: 15


### User Function(s)

In [5]:
def DefinitionSearch(datadic, col, flag=False):
    # initialize variable
    parm =  "r'(?i)" + col + "'" # regex search using ignore case sensitivity
    parm = eval(parm)
    # display
    df_str = datadic.loc[:,['featureName','desc', 'dataType', 'labelSAS', 'COMMENT', 'Information']][datadic.featureName.str.contains(parm)]

    if flag:
        feature = datadic.featureName[datadic.featureName.str.contains(parm)].tolist()
        return feature
    else:
        return df_str


def removeColumn(data, col):
    """
    Remove unwanted columns
    """
    # display removed feature(s)
    print(f"\nRemoved Features:{col}\n")
    # display shape of DataFrame
    print(f"Total rows before: {data.shape[0]:,} & columns: {data.shape[1]:,}")
    
    # remove column
    data.drop(columns=col, axis=1, inplace=True)

    # reset index in place
    data.reset_index(drop=True, inplace=True)

    # display shape of DataFrame
    print(f"Total rows after: {data.shape[0]:,} & columns: {data.shape[1]:,}")

    return data


def removeRowUsingMask(data, removeColLst, colstr):
    # boolean mask
    mask = ~data[colstr].isin(removeColLst)
    
    # apply the mask to keep only rows where 'removeColLst'
    data = data[mask]
    
    # reset the index if needed
    data = data.reset_index(drop=True)

    # disply row removed msg
    print(f"Remove row(s) from df_{colstr} DataFrame.")

    return data


def updateDataDict(data, remove, col="COMMENT"):
    """
    Maintain data dictionary
    """
    # update data dictionary
    idx = data[data.featureName.isin(remove)].index
    # append to exiting data
    data.loc[idx,col] = "**REMOVED 6** - " + data[col]

    # disply update msg
    print(f"Data Dictionary Updated.")

    return data


def removeHouseKeeping(data, removeColLst, dataDict, dataRemove, dataUnknown, dataLabel, dataNew, dataCAN, dataDON, dataBoth):
    """
    Run helper fuction for house keeping
    """
    # update data dictionary (house keeping)
    dataDict = updateDataDict(dataDict, removeColLst)
    
    # remove DataFrame data (house keeping)
    dataRemove = removeRowUsingMask(dataRemove, removeColLst, colstr='remove')
    dataUnknown = removeRowUsingMask(dataUnknown, removeColLst, colstr='unknown')
    dataLabel = removeRowUsingMask(dataLabel, removeColLst, colstr='label')
    dataNew = removeRowUsingMask(dataNew, removeColLst, colstr='new')
    dataCAN = removeRowUsingMask(dataCAN, removeColLst, colstr='can')
    dataDON = removeRowUsingMask(dataDON, removeColLst, colstr='don')
    dataBoth = removeRowUsingMask(dataBoth, removeColLst, colstr='both')
    
    # remove features
    data = removeColumn(data, removeColLst)

    return data, dataDict, dataRemove, dataUnknown, dataLabel, dataNew, dataCAN, dataDON, dataBoth



def datatypeDF(data, display=True):
    # initialize variables for all the column name per each datatype
    boolCol = data.select_dtypes(include=['bool']).columns.tolist()
    catCol = data.select_dtypes(include=['category']).columns.tolist()
    objCol = data.select_dtypes(include=['object']).columns.tolist()
    numCol = data.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if display:
        # display feature counts
        print('Total Data feature count: ', df.shape[1])
        print(f"\nBoolean feature count: {len(boolCol)}")
        print(f"Category feature count: {len(catCol)}")
        print(f"Numeric feature count: {len(numCol)}")
        print(f"Object feature count: {len(objCol)}\n")
        print('Total feature count: ' ,len(boolCol) + len(catCol) + len(objCol) + len(numCol))
    else:
        return boolCol, catCol, objCol, numCol


### Wrangle Data

In [6]:
# remove - values after the transplant Features
removeCol = df_can.can[df_can.can.str.contains('After')].to_list()

# house keeping
df, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both = removeHouseKeeping(df, removeCol, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both)

Data Dictionary Updated.
Remove row(s) from df_remove DataFrame.
Remove row(s) from df_unknown DataFrame.
Remove row(s) from df_label DataFrame.
Remove row(s) from df_new DataFrame.
Remove row(s) from df_can DataFrame.
Remove row(s) from df_don DataFrame.
Remove row(s) from df_both DataFrame.

Removed Features:['ResidencyStateAfter_CAN', 'LifeSupportAfter_ECMO_CAN', 'LifeSupport_ECMOAfter_CAN_difference', 'LifeSupportAfter_IABP_CAN', 'LifeSupport_IABPAfter_CAN_difference', 'IntropesIVAfter_CAN', 'IntropesIVAfter_CAN_difference', 'LifeSupportAfter_PGE_CAN', 'LifeSupport_PGEAfter_CAN_difference', 'LifeSupportAfter_OTHER_CAN', 'LifeSupport_OTHERAfter_CAN_difference', 'VentricularDeviceTypeAfter_CAN', 'VentricularDeviceTypeAfter_CAN_difference', 'VentricularDeviceBrandAfter_CAN', 'VentricularDeviceBrandAfter_CAN_difference', 'FunctionalStatusAfter_CAN', 'FunctionalStatusAfter_CAN_difference', 'FunctionalStatus_FollowUpAfter_CAN_difference', 'PrimaryPaymentSourceAfter_CAN', 'PrimaryPaymentA

##### `AllocationType` UNOS refers to the type of organ allocation system or method used by UNOS to match donated organs with recipients on the transplant waiting list.
- `AllocationType`

In [7]:
df.AllocationType.value_counts()

AllocationType
Local            6015
National         5752
Regional         3503
Foreign Donor      12
Name: count, dtype: int64

In [8]:
print("Total Length: ",len(df.columns.to_list()))
print(sorted(df.columns.to_list()))

Total Length:  227
['AcuteRejectionEpisode_CAN', 'AdmissionDischarge_CAN_difference', 'AgeCalc_DON', 'Age_CAN', 'Age_difference', 'AirwayDehiscencePostTransplant_CAN', 'AllocationType', 'AntibodyResultHBSAB_DON', 'Antibody_HEP_C_DON', 'AntigenBW4_CAN', 'AntigenBW6_CAN', 'AntigenBW_CAN_difference', 'AntigenC1_CAN', 'AntigenC2_CAN', 'AntigenC_CAN_difference', 'AntigenDQ1_CAN', 'AntigenDQ2_CAN', 'AntigenDQ_CAN_difference', 'AntigenDR51_2_CAN', 'AntigenDR51_CAN', 'AntigenDR52_2_CAN', 'AntigenDR52_CAN', 'AntigenDR53_2_CAN', 'AntigenDR53_CAN', 'AntigenDR5_12_CAN_difference', 'AntigenDR5_22_CAN_difference', 'AntigenDR5_32_CAN_difference', 'AntigenHLAcompatibility_A1_CAN', 'AntigenHLAcompatibility_A2_CAN', 'AntigenHLAcompatibility_B1_CAN', 'AntigenHLAcompatibility_B2_CAN', 'AntigenHLAcompatibility_DR1_CAN', 'AntigenHLAcompatibility_DR2_CAN', 'AntigenHistocompatibility_A1_DON', 'AntigenHistocompatibility_A2_DON', 'AntigenHistocompatibility_B1_DON', 'AntigenHistocompatibility_B2_DON', 'AntigenHi

In [9]:
# features skew towards on value or After Transplant infomation or new feature created
removeOrAfterCol = ['AcuteRejectionEpisode_CAN', 'AdmissionDischarge_CAN_difference', 'AirwayDehiscencePostTransplant_CAN','BronchoscopyLeft_DON','BronchoscopyRight_DON','CancerSite_DON',
                    'Citizenship_CAN','Citizenship_DON','Citizenship_difference','CombinedMeds','CreatinineAfter_CAN_difference','DiabetesType_CAN','DiagnosisType_CAN',
                    'DialysisPriorDischarge_CAN','DialysisType_CAN','FollowUpFunctionalStatus_CAN','GraftFailed_CAN','GraftStatus_CAN','HomeState_DON','LastFollowupNumber_CAN',
                    'LengthOfStay_CAN','PrimaryDiagnosisCode_CAN','PrimaryPaymentSource_CAN',
                    'RecipientStatus_CAN','RecoveryTransplant_difference','RejectionTreatmentWithinOneYear_CAN','ResidencyStateAfter_CAN_difference','ResidencyState_CAN',
                    'ResidencyState_difference','Status_CAN','Status_DON','Status_difference','StrokePostTransplant_CAN','TransplantStatus_CAN','TransplantSurvivalDay_CAN',
                    'TransplantType_CAN','VentricularDeviceBrand_CAN','VentricularDeviceType_CAN','WaitListDiagnosisCode_CAN'
                   ]
# features to keep
keepCanCol = ['Age_CAN','AntigenBW4_CAN','AntigenBW6_CAN','AntigenBW_CAN_difference','AntigenC1_CAN','AntigenC2_CAN','AntigenC_CAN_difference','AntigenDQ1_CAN', 'AntigenDQ2_CAN',
              'AntigenDQ_CAN_difference', 'AntigenDR51_2_CAN', 'AntigenDR51_CAN','AntigenDR52_2_CAN', 'AntigenDR52_CAN', 'AntigenDR53_2_CAN','AntigenDR53_CAN',
              'AntigenDR5_12_CAN_difference', 'AntigenDR5_22_CAN_difference','AntigenDR5_32_CAN_difference','AntigenHLAcompatibility_A1_CAN','AntigenHLAcompatibility_A2_CAN',
              'AntigenHLAcompatibility_B1_CAN','AntigenHLAcompatibility_B2_CAN','AntigenHLAcompatibility_DR1_CAN','AntigenHLAcompatibility_DR2_CAN','AtTransplantSurfaceHBVAntibodyTotal_CAN',
              'BMI_CAN','BloodGroup_CAN','CMVStatusAtTransplant_CAN','CalcPanelReactiveAntibody_CPRA_CAN_difference','CerebrovascularDisease_CAN','ChronicSteroidsUse_CAN',
              'CigaretteAbstinence_CAN','CigaretteUse_CAN','Creatinine_CAN','DefibrillatorImplant_CAN','DialysisPrior_CAN','EducationLevel_CAN','EpsteinBarrStatusTransplant_CAN',
              'Ethnicity_CAN','FunctionalStatus_CAN','Gender_CAN','HeartProcedureType_CAN','HeightCm_CAN','Hemodynamics_CO_CAN','Hemodynamics_PA_DIA_CAN','Hemodynamics_PA_MN_CAN',
              'Hemodynamics_PCW_CAN','Hemodynamics_SYS_CAN','Hepatitis_B_CoreAntibody_CAN','Hispanic_CAN','IntropesVasodilators_CO_CAN','IntropesVasodilators_DIA_CAN',
              'InfectionTherapyIV_CAN','IntropesIV_CAN','IntropesVasodilators_MN_CAN', 
              'IntropesVasodilators_PCW_CAN','IntropesVasodilators_SYS_CAN','LifeSupportInhaled_CAN','LifeSupport_CAN','LifeSupport_ECMO_CAN','LifeSupport_IABP_CAN', 
              'LifeSupport_OTHER_CAN','LifeSupport_PGE_CAN','PacemakerPostTransplant_CAN','Peak_CPRA_CAN','PreTranspantHCV_NAT_CAN','PreTransplantMedicalCondition_CAN',
              'PreTransplantResultHIV_NAT_CAN','PreviousMalignancy_CAN','PreviousTransplantAnyOrgan_CAN','PreviousTransplantNumber_CAN','PreviousTransplantSameOrgan_CAN',
              'PriorCardiacSurgeryType_CAN','PriorCardiacSurgery_CAN','PriorLungSurgery_CAN','Recent_CPRA_CAN',
              'SerostatusStatus_HEP_C_CAN','SurfaceAntigenHEP_B_CAN','TotalBilirubin_CAN','TotalDayWaitList_CAN','Transfusion_CAN','TransplantSerostatusHIV_CAN','Ventilator_CAN',
              'WeightKg_CAN','WorkIncome_CAN'
             ]
keepDonCol = ['AgeCalc_DON','AntibodyResultHBSAB_DON','Antibody_HEP_C_DON','AntigenHistocompatibility_A1_DON','AntigenHistocompatibility_A2_DON','AntigenHistocompatibility_B1_DON',
              'AntigenHistocompatibility_B2_DON','AntigenHistocompatibility_DR1_DON','AntigenHistocompatibility_DR2_DON','Antihypertensive_DON','ArginnieManagement_DON','BMI_DON',
              'Biopsy_DON','BloodGroup_DON','BloodInfectionSource_DON','BloodPH_DON','BloodUreaNitrogenLevel_DON','CardiacArrest_DON','CauseOfDeath_DON','ChestXray_DON',
              'CigaretteHistory_DON','ClinicalInfection_DON','CocaineUse_DON','CoronaryAngiogram_DON','Creatinine_DON','DeathCircumstance_DON','DeathMechanism_DON','DeceasedRetyped_DON',
              'DiabetesHistory_DON','Diabetes_DON','Diuretics_DON','EpsteinBarr_IGG_DON','EpsteinBarr_IGM_DON','Ethnicity_DON','ExtracanialCancer_DON','Gender_DON','HeavyAlcoholUse_DON',
              'HeightCm_DON','Hematocrit_DON','HeparinManagement_DON','Hepatitis_B_CoreAntibody_DON','HistoryCancer_DON','Hypertension_DON','InotropicMedication_DON','InsulinManagement_DON',
              'IntracanialCancer_DON','IschemicTimeHours_DON','KidneyAllocation_DON','Level_SGOT_AST_DON','LV_EjectionFractionMedthod_DON',
              'LV_EjectionFractionPercent_DON','Level_SGOT_ALT_DON','LungPO2_DON','LungPO2_FIO2_DON','MyocardialInfarction_DON','NonHeartBeating_DON','OtherDrugUse_DON',
              'OtherInfectionSource_DON','PCO2_DON', 'PastCocaineUse_DON','PastOtherDrugUse_DON','PulmCath_DON','PulmonaryInfection_DON','ResultHBV_NAT_DON','ResultHCV_NAT_DON',
              'ResultHIV_NAT_DON','Result_RPR_VDRL_DON', 'RiskHIV_DON','SerologyAntiCMV_DON','SkinCancer_DON','SteroidsUse_DON','SurfaceAntigenHEP_B_DON','SynthicAntiDiureticHormone_DON',
              'Tatoos_DON','Thyroxine_T4_DON', 'TotalBilirubin_DON','TransfusionNumber_DON','Triiodothyronine_T3_DON','UrineInfection_DON','UrinePortein_DON','Vasodilator_DON','WeightKg_DON'
             ]
keepBothCol = ['Age_difference','AllocationType', 'BMI_difference','BloodGroupMatchLevel','BloodGroup_difference','Cigarette_difference','Creatinine_difference','CrossMatch',
               'Dialysis_CAN_difference','DistanceFrom_DON_HospitaltoTXCenter','Ethnicity_difference','Gender_difference','HeightCm_difference','MismatchLevel_B_LOCUS',
               'MismatchLevel_DR_LOCUS','MismatchLevel_HLA','MismatchLevel_LOCUS','TotalBilirubin_difference','TransplantRegion','WeightKg_difference'
              ]

# display total length
print("Total Length: ", len(removeOrAfterCol + keepCanCol + keepDonCol + keepBothCol))

Total Length:  227


#### Consoldate Features

In [10]:
# new Features
df['Cancer_DON'] = df['CancerSite_DON'].apply(lambda x: 'YES' if x not in ['UNKNOWN', 'NO'] else x)
df['Diabetes_CAN'] = df['DiabetesType_CAN'].apply(lambda x: 'Yes' if x not in ['Unknown', 'No'] else x)
df['BronchoscopyNewLeft_DON'] = df['BronchoscopyLeft_DON'].apply(lambda x: 'Unknown' \
                        if x in ['Unknown','Bronchoscopy Results, Unknown','No Bronchoscopy'] \
                        else 'Normal' if x == 'Bronchoscopy Results normal' \
                        else 'Abnormal')
df['BronchoscopyNewRight_DON'] = df['BronchoscopyRight_DON'].apply(lambda x: 'Unknown' \
                        if x in ['Unknown','Bronchoscopy Results, Unknown','No Bronchoscopy'] \
                        else 'Normal' if x == 'Bronchoscopy Results normal' \
                        else 'Abnormal')

# add to list
keepCanCol.extend(['Diabetes_CAN'])
keepDonCol.extend(['Cancer_DON','BronchoscopyNewLeft_DON','BronchoscopyNewRight_DON'])

#### Remove Features

In [11]:
# house keeping
df, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both = removeHouseKeeping(df, removeOrAfterCol, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both)

Data Dictionary Updated.
Remove row(s) from df_remove DataFrame.
Remove row(s) from df_unknown DataFrame.
Remove row(s) from df_label DataFrame.
Remove row(s) from df_new DataFrame.
Remove row(s) from df_can DataFrame.
Remove row(s) from df_don DataFrame.
Remove row(s) from df_both DataFrame.

Removed Features:['AcuteRejectionEpisode_CAN', 'AdmissionDischarge_CAN_difference', 'AirwayDehiscencePostTransplant_CAN', 'BronchoscopyLeft_DON', 'BronchoscopyRight_DON', 'CancerSite_DON', 'Citizenship_CAN', 'Citizenship_DON', 'Citizenship_difference', 'CombinedMeds', 'CreatinineAfter_CAN_difference', 'DiabetesType_CAN', 'DiagnosisType_CAN', 'DialysisPriorDischarge_CAN', 'DialysisType_CAN', 'FollowUpFunctionalStatus_CAN', 'GraftFailed_CAN', 'GraftStatus_CAN', 'HomeState_DON', 'LastFollowupNumber_CAN', 'LengthOfStay_CAN', 'PrimaryDiagnosisCode_CAN', 'PrimaryPaymentSource_CAN', 'RecipientStatus_CAN', 'RecoveryTransplant_difference', 'RejectionTreatmentWithinOneYear_CAN', 'ResidencyStateAfter_CAN_di

#### Wrangle

In [12]:
# find all binary categories exclude boolean
catBinary = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype != 'bool']

# display
print(catBinary)

['Gender_CAN', 'LifeSupport_ECMO_CAN', 'LifeSupport_IABP_CAN', 'IntropesIV_CAN', 'LifeSupport_PGE_CAN', 'LifeSupport_OTHER_CAN', 'Hispanic_CAN', 'Ventilator_CAN', 'LifeSupportInhaled_CAN', 'PreviousTransplantSameOrgan_CAN', 'PreviousTransplantAnyOrgan_CAN', 'Gender_DON', 'BloodInfectionSource_DON', 'PulmonaryInfection_DON', 'UrineInfection_DON', 'ResultHIV_NAT_DON', 'KidneyAllocation_DON']


In [13]:
df[catBinary].describe().T

,count,unique,top,freq
Gender_CAN,15282,2,M,11175
LifeSupport_ECMO_CAN,15282,2,0,14858
LifeSupport_IABP_CAN,15282,2,0,13473
IntropesIV_CAN,15282,2,0,10228
LifeSupport_PGE_CAN,15282,2,0,15275
LifeSupport_OTHER_CAN,15282,2,0,14562
Hispanic_CAN,15282,2,0,13826
Ventilator_CAN,15282,2,0,15039
LifeSupportInhaled_CAN,15282,2,0,15247
PreviousTransplantSameOrgan_CAN,15282,2,N,14820


In [14]:
# define the mapping
mapping = {
    'M': True,
    'F': False,    
      1: True,
      0: False,
    'Y': True,
    'N': False,
    'Negative': False,
    'Positive': True
}

# apply the mapping to each relevant column using map
for col in catBinary:
    df[col] = df[col].map(mapping)

# change datatype
df[catBinary] = df[catBinary].astype('bool')

# display head
df[catBinary].head()

,Gender_CAN,LifeSupport_ECMO_CAN,LifeSupport_IABP_CAN,IntropesIV_CAN,LifeSupport_PGE_CAN,LifeSupport_OTHER_CAN,Hispanic_CAN,Ventilator_CAN,LifeSupportInhaled_CAN,PreviousTransplantSameOrgan_CAN,PreviousTransplantAnyOrgan_CAN,Gender_DON,BloodInfectionSource_DON,PulmonaryInfection_DON,UrineInfection_DON,ResultHIV_NAT_DON,KidneyAllocation_DON
0,True,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False


In [15]:
# # house keeping # remove unwanted columns
# df, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both = removeHouseKeeping(df, removeCol, df_dict, df_remove, df_unknown, df_label, df_new, df_can, df_don, df_both)

### Determine DataFrame Datatypes¶

In [15]:
# display msg only
datatypeDF(df)
# initialize list variables for feature names
boolCol, catCol, objCol, numCol = datatypeDF(df, False)

Total Data feature count:  192

Boolean feature count: 28
Category feature count: 98
Numeric feature count: 62
Object feature count: 4

Total feature count:  192


In [16]:
df[objCol].describe().T

,count,unique,top,freq
Cancer_DON,15282,3,NO,14961
Diabetes_CAN,15282,3,No,10768
BronchoscopyNewLeft_DON,15282,3,Unknown,8239
BronchoscopyNewRight_DON,15282,3,Unknown,8298


In [17]:
# change datatype to category
df[objCol] = df[objCol].astype('category')

In [18]:
# display msg only
datatypeDF(df)
# initialize list variables for feature names
boolCol, catCol, objCol, numCol = datatypeDF(df, False)

Total Data feature count:  192

Boolean feature count: 28
Category feature count: 102
Numeric feature count: 62
Object feature count: 0

Total feature count:  192


In [19]:
df[catCol].head()

,BloodGroup_CAN,EducationLevel_CAN,FunctionalStatus_CAN,CerebrovascularDisease_CAN,PreviousMalignancy_CAN,DefibrillatorImplant_CAN,IntropesVasodilators_SYS_CAN,IntropesVasodilators_DIA_CAN,IntropesVasodilators_MN_CAN,IntropesVasodilators_PCW_CAN,IntropesVasodilators_CO_CAN,CigaretteUse_CAN,CigaretteAbstinence_CAN,PriorCardiacSurgery_CAN,PriorCardiacSurgeryType_CAN,LifeSupport_CAN,Ethnicity_CAN,TransplantRegion,WorkIncome_CAN,AntigenBW4_CAN,AntigenBW6_CAN,PreTransplantMedicalCondition_CAN,InfectionTherapyIV_CAN,PriorLungSurgery_CAN,PacemakerPostTransplant_CAN,ChronicSteroidsUse_CAN,Transfusion_CAN,Hepatitis_B_CoreAntibody_CAN,SurfaceAntigenHEP_B_CAN,AtTransplantSurfaceHBVAntibodyTotal_CAN,CMVStatusAtTransplant_CAN,TransplantSerostatusHIV_CAN,SerostatusStatus_HEP_C_CAN,EpsteinBarrStatusTransplant_CAN,PreTransplantResultHIV_NAT_CAN,PreTranspantHCV_NAT_CAN,DeceasedRetyped_DON,CrossMatch,MismatchLevel_LOCUS,MismatchLevel_B_LOCUS,MismatchLevel_DR_LOCUS,MismatchLevel_HLA,PastCocaineUse_DON,Ethnicity_DON,Hepatitis_B_CoreAntibody_DON,SurfaceAntigenHEP_B_DON,BloodGroup_DON,HeavyAlcoholUse_DON,Antibody_HEP_C_DON,NonHeartBeating_DON,Antihypertensive_DON,Diuretics_DON,SteroidsUse_DON,Triiodothyronine_T3_DON,Thyroxine_T4_DON,Vasodilator_DON,Result_RPR_VDRL_DON,ClinicalInfection_DON,CocaineUse_DON,OtherDrugUse_DON,ExtracanialCancer_DON,CigaretteHistory_DON,Hypertension_DON,IntracanialCancer_DON,HistoryCancer_DON,DiabetesHistory_DON,Diabetes_DON,PastOtherDrugUse_DON,SkinCancer_DON,SerologyAntiCMV_DON,CauseOfDeath_DON,SynthicAntiDiureticHormone_DON,DeathCircumstance_DON,DeathMechanism_DON,HeparinManagement_DON,ArginnieManagement_DON,InsulinManagement_DON,ResultHBV_NAT_DON,ResultHCV_NAT_DON,BloodGroupMatchLevel,DialysisPrior_CAN,AllocationType,HeartProcedureType_CAN,UrinePortein_DON,CardiacArrest_DON,MyocardialInfarction_DON,LV_EjectionFractionMedthod_DON,CoronaryAngiogram_DON,Biopsy_DON,Tatoos_DON,AntibodyResultHBSAB_DON,EpsteinBarr_IGG_DON,EpsteinBarr_IGM_DON,RiskHIV_DON,InotropicMedication_DON,TransfusionNumber_DON,PulmCath_DON,ChestXray_DON,Cancer_DON,Diabetes_CAN,BronchoscopyNewLeft_DON,BronchoscopyNewRight_DON
0,O,HIGH SCHOOL (9-12) or GED,70% - Cares for self: unable to carry on normal activity or active work,N,N,Y,N,N,N,N,N,Y,Unknown duration,N,Unknown,Y,"White, Non-Hispanic",1,N,0,0,Not Hospitalized,N,N,N,N,Y,Negative,Negative,Negative,Positive,Negative,Negative,Positive,Unknown,Unknown,N,Y,999,999,999,999,N,Hispanic/Latino,Negative,Negative,O,N,Negative,N,N,Y,Y,N,N,N,Negative,Y,U,Y,N,N,N,N,N,No,N,Y,N,Positive,HEAD TRAUMA,N,HOMICIDE,GUNSHOT WOUND,Y,Y,N,Negative,Negative,Identical,N,Local,Orthotopic Bicaval,N,N,N,Echo,No,Needle,Y,Not Done,Positive,Negative,N,N,NONE,N,Abnormal-both,NO,Yes,Abnormal,Abnormal
1,A,HIGH SCHOOL (9-12) or GED,70% - Cares for self: unable to carry on normal activity or active work,N,N,Y,N,N,N,N,N,N,Unknown duration,N,Unknown,Y,"Black, Non-Hispanic",11,N,0,0,Not Hospitalized,Y,N,N,N,N,Negative,Negative,Negative,Positive,Negative,Negative,Positive,Unknown,Unknown,N,Y,999,999,999,999,Y,"White, Non-Hispanic",Negative,Negative,A1,N,Negative,N,Y,Y,Y,N,N,Y,Negative,Y,N,U,N,N,N,N,N,"Yes, >10 years",Y,N,N,Positive,ANOXIA,N,DEATH FROM NATURAL CAUSES,CARDIOVASCULAR,N,N,Y,Negative,Negative,Identical,N,Regional,Orthotopic Traditional,N,N,N,Echo,"Yes, normal",Needle,Y,Not Done,Positive,Negative,N,N,NONE,N,Normal,NO,No,Normal,Normal
2,A,HIGH SCHOOL (9-12) or GED,"30% - Severely disabled: hospitalization is indicated, death not imminent",N,N,Y,N,N,N,N,N,N,Unknown duration,Y,"Valve Replace/Repair; Other, specify",Y,"White, Non-Hispanic",5,Y,0,0,Not Hospitalized,N,N,N,N,N,Negative,Negative,Negative,Positive,Negative,Negative,Positive,Not Done,Not Done,Y,Y,2,2,2,6,N,"Amer Ind/Alaska Native, Non-Hispanic",Negative,Negative,O,N,Negative,N,Y,Y,Y,N,N,Y,Negative,Y,U,U,N,N,N,N,N,No,N,N,N,Positive,ANOXIA,N,SUICIDE,DRUG INTOXICATION,Y,Y,Y,Negative,Negative,Compatible,N,Local,Orthotopic Bicaval,N,N,N,Echo,"Yes, normal",Needle,N,Not Done,No

### Encoding

In [29]:
df.FunctionalStatus_CAN.value_counts()

FunctionalStatus_CAN
20% - Very sick, hospitalization necessary: active treatment necessary       1867
40% - Disabled: requires special care and assistance                         1510
60% - Requires occasional assistance but is able to care for needs           1126
70% - Cares for self: unable to carry on normal activity or active work      1034
50% - Requires considerable assistance and frequent medical care              933
30% - Severely disabled: hospitalization is indicated, death not imminent     896
80% - Normal activity with effort: some symptoms of disease                   637
10% - Moribund, fatal processes progressing rapidly                           191
90% - Able to carry on normal activity: minor symptoms of disease             167
Unknown                                                                       164
100% - Normal, no complaints, no evidence of disease                           32
Name: count, dtype: int64

In [24]:
print(sorted(catCol))

['AllocationType', 'AntibodyResultHBSAB_DON', 'AntigenBW4_CAN', 'AntigenBW6_CAN', 'Antihypertensive_DON', 'ArginnieManagement_DON', 'AtTransplantSurfaceHBVAntibodyTotal_CAN', 'Biopsy_DON', 'BloodGroupMatchLevel', 'BloodGroup_CAN', 'BloodGroup_DON', 'BronchoscopyNewLeft_DON', 'BronchoscopyNewRight_DON', 'CMVStatusAtTransplant_CAN', 'Cancer_DON', 'CardiacArrest_DON', 'CauseOfDeath_DON', 'CerebrovascularDisease_CAN', 'ChestXray_DON', 'ChronicSteroidsUse_CAN', 'CigaretteAbstinence_CAN', 'CigaretteHistory_DON', 'ClinicalInfection_DON', 'CocaineUse_DON', 'CoronaryAngiogram_DON', 'DeathCircumstance_DON', 'DeathMechanism_DON', 'DefibrillatorImplant_CAN', 'DiabetesHistory_DON', 'Diabetes_CAN', 'Diabetes_DON', 'DialysisPrior_CAN', 'Diuretics_DON', 'EducationLevel_CAN', 'EpsteinBarrStatusTransplant_CAN', 'EpsteinBarr_IGG_DON', 'EpsteinBarr_IGM_DON', 'Ethnicity_CAN', 'Ethnicity_DON', 'ExtracanialCancer_DON', 'FunctionalStatus_CAN', 'HeartProcedureType_CAN', 'HeavyAlcoholUse_DON', 'HeparinManagemen

In [15]:
# ordinal list
ordinalCol = ['DiabetesHistory_DON', 'EducationLevel_CAN', 'InitialFunctionalStatus_DON', 'FunctionalStatus_CAN', 'PreTransplantMedicalCondition_CAN']
# nominal list
nominalCol =  pd.Series(catCol)[~pd.Series(catCol).isin(ordinalCol)].to_list()

# ordinal list
ordering = ['Unknown', '100% - Normal, no complaints, no evidence of disease', '90% - Able to carry on normal activity: minor symptoms of disease', 
            '80% - Normal activity with effort: some symptoms of disease', '70% - Cares for self: unable to carry on normal activity or active work',
            '60% - Requires occasional assistance but is able to care for needs', '50% - Requires considerable assistance and frequent medical care',
            '40% - Disabled: requires special care and assistance', '30% - Severely disabled: hospitalization is indicated, death not imminent',
            '20% - Very sick, hospitalization necessary: active treatment necessary', '10% - Moribund, fatal processes progressing rapidly'
           ]


# apply OrdinalEncoder - FunctionalStatus_CAN
encoder = OrdinalEncoder(categories=[ordering])
df['FunctionalStatus_CAN'] = encoder.fit_transform(df[['FunctionalStatus_CAN']])

# apply OrdinalEncoder - InitialFunctionalStatus_DON
encoder = OrdinalEncoder(categories=[ordering])
df['InitialFunctionalStatus_DON'] = encoder.fit_transform(df[['InitialFunctionalStatus_DON']])


# ordinal list
ordering = ['Unknown', 'Not Hospitalized', 'Hospitalized Not in ICU', 'In Intensive Care Unit']

# apply OrdinalEncoder - PreTransplantMedicalCondition_CAN
encoder = OrdinalEncoder(categories=[ordering])
df['PreTransplantMedicalCondition_CAN'] = encoder.fit_transform(df[['PreTransplantMedicalCondition_CAN']])


# DiabetesHistory_DON
ordering = ['Unknown','No','Yes, Duration Unknown','Yes, 0-5 years','Yes, >10 years','Yes, 6-10 years']

# apply OrdinalEncoder - DiabetesHistory_DON
encoder = OrdinalEncoder(categories=[ordering])
df['DiabetesHistory_DON'] = encoder.fit_transform(df[['DiabetesHistory_DON']])


# EducationLevel_CAN
ordering = ['UNKNOWN','NONE','GRADE SCHOOL (0-8)','HIGH SCHOOL (9-12) or GED','ATTENDED COLLEGE/TECHNICAL SCHOOL','ASSOCIATE/BACHELOR DEGREE','POST-COLLEGE GRADUATE DEGREE']

# apply OrdinalEncoder - EducationLevel_CAN
encoder = OrdinalEncoder(categories=[ordering])
df['EducationLevel_CAN'] = encoder.fit_transform(df[['EducationLevel_CAN']])

df[ordinalCol].head(10)

,DiabetesHistory_DON,EducationLevel_CAN,InitialFunctionalStatus_DON,FunctionalStatus_CAN,FunctionalStatus_DON,PreTransplantMedicalCondition_CAN
0,1.0,3.0,1.0,8.0,7.0,1.0
1,1.0,5.0,1.0,4.0,6.0,1.0
2,1.0,3.0,4.0,6.0,9.0,2.0
3,1.0,5.0,3.0,2.0,4.0,1.0
4,3.0,6.0,0.0,2.0,2.0,1.0
5,1.0,3.0,10.0,7.0,7.0,1.0
6,1.0,6.0,0.0,3.0,5.0,1.0
7,1.0,5.0,0.0,7.0,9.0,2.0
8,1.0,4.0,3.0,5.0,3.0,1.0
9,1.0,3.0,3.0,8.0,9.0,3.0


In [16]:
print(sorted(catCol))

['AllocationType', 'AntibodyResultHBSAB_DON', 'Antibody_HEP_C_DON', 'Antihypertensive_DON', 'ArginnieManagement_DON', 'AtTransplantSurfaceHBVAntibodyTotal_DON', 'Biopsy_DON', 'BloodGroupMatchLevel', 'BloodInfectionSource_DON', 'BronchoscopyLeft_DON', 'BronchoscopyRight_DON', 'CMVStatusAtTransplant', 'CancerSite_DON', 'CardiacArrest_DON', 'CauseOfDeath_DON', 'CerebrovascularDisease_CAN', 'ChestXray_DON', 'ChronicSteroidsUse_DON', 'ClinicalInfection_DON', 'CocaineUse_DON', 'CoronaryAngiogram_DON', 'CrossMatch', 'DeathCircumstance_DON', 'DeathMechanism_DON', 'DefibrillatorImplant_CAN', 'DiabetesHistory_DON', 'Diabetes_CAN', 'Diabetes_DON', 'DiagnosisType', 'DialysisType_CAN', 'Dialysis_CAN', 'Dialysis_DON', 'Diuretics_DON', 'EducationLevel_CAN', 'EpsteinBarrStatusTransplant', 'EpsteinBarr_IGG_DON', 'EpsteinBarr_IGM_DON', 'ExtracanialCancer_DON', 'FunctionalStatus_CAN', 'FunctionalStatus_DON', 'HeartProcedureType', 'HeavyAlcoholUse_DON', 'HeparinManagement_DON', 'Hepatitis_B_CoreAntibody',

### Random Forest Survival Analysis
- A **Random Forest Survival Analysis** is an extension of the Random Forest algorithm adapted to work with survival data, commonly used in medical or reliability studies where the outcome is the time until an event occurs (e.g., time until death, failure of a machine). It can model the relationship between covariates (features) and the survival time. Survival analysis typically deals with censored data, meaning that, in some instances, the event has yet to occur by the end of the study period. Random Forest for Survival is designed to handle such data.

In [17]:
df[catCol].head()

,WaitListDiagnosisCode_CAN,EducationLevel_CAN,FunctionalStatus_CAN,PrimaryDiagnosisCode_CAN,Diabetes_CAN,DialysisType_CAN,CerebrovascularDisease_CAN,PreviousMalignancy_CAN,DefibrillatorImplant_CAN,FunctionalStatus_DON,PreTransplantMedicalCondition_CAN,InfectionTherapyIV,PriorLungSurgery_DON,ChronicSteroidsUse_DON,Transfusion,Hepatitis_B_CoreAntibody,SurfaceAntigenHEP_B,AtTransplantSurfaceHBVAntibodyTotal_DON,CMVStatusAtTransplant,TransplantSerostatusHIV_DON,SerostatusStatus_HEP_C,EpsteinBarrStatusTransplant,PreTransplantResultHIV_NAT,PreTranspantHCV_NAT_DON,ResultHBV_NAT,TransplantStatus,InitialFunctionalStatus_DON,CrossMatch,PreviousTransplantSameOrgan,PreviousTransplantAnyOrgan,PastCocaineUse_DON,Hepatitis_B_CoreAntibody_DON,SurfaceAntigenHEP_B_DON,HeavyAlcoholUse_DON,Antibody_HEP_C_DON,NonHeartBeating_DON,Antihypertensive_DON,BloodInfectionSource_DON,Diuretics_DON,SteroidsUse_DON,Triiodothyronine_T3_DON,Thyroxine_T4_DON,PulmonaryInfection_DON,UrineInfection_DON,Vasodilator_DON,Result_RPR_VDRL_DON,ClinicalInfection_DON,CocaineUse_DON,OtherDrugUse_DON,ExtracanialCancer_DON,CancerSite_DON,Hypertension_DON,IntracanialCancer_DON,HistoryCancer_DON,DiabetesHistory_DON,Diabetes_DON,PastOtherDrugUse_DON,SkinCancer_DON,CauseOfDeath_DON,SynthicAntiDiureticHormone_DON,DeathCircumstance_DON,DeathMechanism_DON,HeparinManagement_DON,ArginnieManagement_DON,InsulinManagement_DON,ResultHBV_NAT_DON,ResultHIV_NAT_DON,ResultHIV_NAT,TransplantType,BloodGroupMatchLevel,DiagnosisType,AllocationType,VentilatorySupport,HeartProcedureType,UrinePortein_DON,CardiacArrest_DON,MyocardialInfarction_DON,LV_EjectionFractionMedthod_DON,CoronaryAngiogram_DON,Biopsy_DON,Tatoos_DON,AntibodyResultHBSAB_DON,EpsteinBarr_IGG_DON,EpsteinBarr_IGM_DON,RiskHIV_DON,KidneyAllocation_DON,InotropicMedication_DON,TranfusionNumber_DON,PulmCath_DON,BronchoscopyLeft_DON,BronchoscopyRight_DON,ChestXray_DON,Dialysis_DON,Dialysis_CAN
0,DILATED MYOPATHY: POST PARTUM,3.0,8.0,DILATED MYOPATHY: OTHER SPECIFY,No,No dialysis,N,N,Y,7.0,1.0,N,N,N,N,Negative,Negative,Negative,Positive,Negative,Negative,Positive,Not Done,Not Done,Not Done,0,1.0,Y,N,N,N,Negative,Negative,N,Negative,N,Y,0,Y,Y,N,N,1,0,Y,Negative,Y,U,U,N,NO,N,N,N,1.0,N,N,N,ANOXIA,N,SUICIDE,DRUG INTOXICATION,Y,Y,Y,Negative,Negative,Negative,Orthotopic,Compatible,DILATED MYOPATHY: OTHER SPECIFY,Local,N,Orthotopic Bicaval,N,N,N,Echo,"Yes, normal",Needle,N,Not Done,Not Done,Not Done,N,0,N,1 - 5,N,Bronchoscopy Results normal,Bronchoscopy Results normal,Abnormal-both,N,N
1,DILATED MYOPATHY: ISCHEMIC,5.0,4.0,DILATED MYOPATHY: ISCHEMIC,No,No dialysis,N,N,N,6.0,1.0,N,N,N,N,Negative,Negative,Negative,Negative,Negative,Negative,Positive,Unknown,Unknown,Unknown,0,1.0,Y,N,N,N,Negative,Negative,N,Negative,N,Y,0,Y,Y,N,Y,1,1,Y,Negative,Y,U,U,N,NO,N,N,N,1.0,N,N,N,CEREBROVASCULAR/STROKE,N,DEATH FROM NATURAL CAUSES,INTRACRANIAL HEMORRHAGE/STROKE,Y,Y,N,Negative,Negative,Negative,Orthotopic,Identical,DILATED MYOPATHY: ISCHEMIC,Local,N,Orthotopic Bicaval,N,N,N,Echo,No,Needle,N,Not Done,Positive,Negative,N,0,N,NONE,N,Unknown,Unknown,Abnormal-both,N,N
2,DILATED MYOPATHY: ISCHEMIC,3.0,6.0,DILATED MYOPATHY: ISCHEMIC,No,No dialysis,N,N,Y,9.0,2.0,N,N,N,N,Negative,Negative,Negative,Negative,Negative,Positive,Positive,Unknown,Unknown,Unknown,0,4.0,Y,N,N,Y,Negative,Negative,N,Positive,N,Y,0,N,N,N,Y,0,0,Y,Negative,N,N,Y,N,NO,N,N,N,1.0,N,Y,N,HEAD TRAUMA,N,SUICIDE,GUNSHOT WOUND,Y,Y,Y,Negative,Positive,Negative,Orthotopic,Identical,DILATED MYOPATHY: ISCHEMIC,Local,Y,Orthotopic Bicaval,N,N,N,Echo,No,Needle,Y,Not Done,Positive,Negative,Y,0,Y,1 - 5,N,Unknown,Unknown,Normal,N,N
3,DILATED MYOPATHY: IDIOPATHIC,5.0,2.0,OTHER,No,No dialysis,N,N,Y,4.0,1.0,N,N,N,N,Negative,Negative,Negative,Positive,Negative,Negative,Positive,Unknown,Unknown,Unknown,0,3.0,Y,N,N,N,Negative,Negative,N,Negative,N,N,0,Y,Y,N,Y,1,0,N,Negative,Y,U,U,N,NO,Y,N,N,1.0,N,N,N,ANOXIA,N,DEATH FROM NATURAL CAUSES,SEIZURE,Y,N,N,Negative,Negative,Negative,Orthotopic,Identical,DILATED MYOPATHY: ISCHEMIC,Re

In [18]:
# find binary columns by checking for exactly 2 unique values
binaryCol = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype != 'bool']

# display
print(sorted(binaryCol))

['Antibody_HEP_C_DON', 'BloodInfectionSource_DON', 'CrossMatch', 'KidneyAllocation_DON', 'NonHeartBeating_DON', 'PreviousMalignancy_CAN', 'PreviousTransplantAnyOrgan', 'PreviousTransplantSameOrgan', 'PulmCath_DON', 'PulmonaryInfection_DON', 'ResultHIV_NAT', 'ResultHIV_NAT_DON', 'Thyroxine_T4_DON', 'TransplantStatus', 'UrineInfection_DON']


In [19]:
# describe
df[binaryCol].describe().T

,count,unique,top,freq
PreviousMalignancy_CAN,8726,2,N,7904
TransplantStatus,8726,2,0,7586
CrossMatch,8726,2,Y,8460
PreviousTransplantSameOrgan,8726,2,N,8470
PreviousTransplantAnyOrgan,8726,2,N,8448
Antibody_HEP_C_DON,8726,2,Negative,7903
NonHeartBeating_DON,8726,2,N,8529
BloodInfectionSource_DON,8726,2,0,7824
Thyroxine_T4_DON,8726,2,Y,5321
PulmonaryInfection_DON,8726,2,1,6373


In [20]:
# apply the conversion using map()
df['PreviousMalignancy_CAN'] = df['PreviousMalignancy_CAN'].map({'N': False, 'Y': True})
df['TransplantStatus'] = df['TransplantStatus'].map({0: False, 1: True})
df['CrossMatch'] = df['CrossMatch'].map({'N': False, 'Y': True})
df['PreviousTransplantSameOrgan'] = df['PreviousTransplantSameOrgan'].map({'N': False, 'Y': True})
df['PreviousTransplantAnyOrgan'] = df['PreviousTransplantAnyOrgan'].map({'N': False, 'Y': True})
df['Antibody_HEP_C_DON'] = df['Antibody_HEP_C_DON'].map({'Negative': False, 'Positive': True})
df['NonHeartBeating_DON'] = df['NonHeartBeating_DON'].map({'N': False, 'Y': True})
df['BloodInfectionSource_DON'] = df['BloodInfectionSource_DON'].map({0: False, 1: True})
df['Thyroxine_T4_DON'] = df['Thyroxine_T4_DON'].map({'N': False, 'Y': True})
df['PulmonaryInfection_DON'] = df['PulmonaryInfection_DON'].map({0: False, 1: True})
df['UrineInfection_DON'] = df['UrineInfection_DON'].map({0: False, 1: True})
df['ResultHIV_NAT_DON'] = df['ResultHIV_NAT_DON'].map({'Negative': False, 'Positive': True})
df['ResultHIV_NAT'] = df['ResultHIV_NAT'].map({'Negative': False, 'Positive': True})
df['KidneyAllocation_DON'] = df['KidneyAllocation_DON'].map({0: False, 1: True})
df['PulmCath_DON'] = df['PulmCath_DON'].map({'N': False, 'Y': True})

In [21]:
# describe
df[binaryCol].describe().T

,count,unique,top,freq
PreviousMalignancy_CAN,8726,2,False,7904
TransplantStatus,8726,2,False,7586
CrossMatch,8726,2,True,8460
PreviousTransplantSameOrgan,8726,2,False,8470
PreviousTransplantAnyOrgan,8726,2,False,8448
Antibody_HEP_C_DON,8726,2,False,7903
NonHeartBeating_DON,8726,2,False,8529
BloodInfectionSource_DON,8726,2,False,7824
Thyroxine_T4_DON,8726,2,True,5321
PulmonaryInfection_DON,8726,2,True,6373


In [22]:
# determine length
len(sorted(set(catCol) - set(binaryCol))), len(catCol)
# initialize new list for non binary list
newCatCol = list((set(catCol) - set(binaryCol)))

print(f"Before removing ordinal columns: {len(newCatCol)}")
print(f"Orginal Columns: {len(ordinalCol)}")
# remove ordinal columns
newCatCol = list(set(newCatCol) - set(ordinalCol))

# check length
print(f"After removing ordinal columns: {len(newCatCol)}")

Before removing ordinal columns: 79
Orginal Columns: 6
After removing ordinal columns: 73


In [23]:
df[newCatCol].describe().T

,count,unique,top,freq
InsulinManagement_DON,8726,3,Y,4545
EpsteinBarr_IGM_DON,8726,4,Negative,6655
DefibrillatorImplant_CAN,8726,3,Y,6385
Dialysis_DON,8726,3,N,8296
DeathMechanism_DON,8726,12,BLUNT INJURY,2173
Biopsy_DON,8726,3,Needle,8721
Vasodilator_DON,8726,3,N,7064
Diabetes_DON,8726,3,N,8318
BronchoscopyLeft_DON,8726,8,Unknown,4470
WaitListDiagnosisCode_CAN,8726,32,DILATED MYOPATHY: IDIOPATHIC,3092
